In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [4]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

In [6]:
from src.et_blue.exporting_utils import process_et_blue

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [13]:
YEAR = 2021

PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/ET_Landsat_decadal_canton_v2"
PATHT_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_{YEAR}_dekadal_from_WaPOR_10m"

ET_GREEN_BAND_NAME = "ET_green"
ET_BAND_NAME = "downscaled"
SCALING_FACTOR = 10  # scaling factor for the ET band to convert it back to float
ET_BAND_RESOLUTION = 10  # in meters
NUMBER_OF_IMAGES = 36
TEMPORAL_RESOLUTION = "dekadal"


ET_BLUE_RAW_PATH = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_{YEAR}_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"

## 1. Load Assets

In [14]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [15]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)

Dates of images in the collection:
2021-04-01
2021-04-11
2021-04-21
2021-06-11
2021-06-21
2021-07-01
2021-07-21
2021-08-11
2021-09-01
2021-09-21
2021-10-01
2021-04-01
2021-04-11
2021-04-21
2021-05-01
2021-06-11
2021-06-21
2021-07-21
2021-08-11
2021-09-01
2021-09-21
2021-10-01


In [16]:
et_green = (
    ee.ImageCollection(PATHT_TO_ET_GREEN)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

et_green_list = et_green.toList(et_collection.size())

# Sanity checks:
print_collection_dates(et_green)

Dates of images in the collection:


In [ ]:
for i in range(time_steps):
    # Get time step information
    time_step_name, _ = get_time_step_info(i, time_step_type)

    # Process ET images
    et_image = ee.Image(et_collection_list.get(i))
    et_green = ee.Image(et_green_list.get(i))
    et_blue = compute_et_blue(et_image, et_green)
    et_blue = back_to_int(et_blue, 100)